In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
from transformers import AutoImageProcessor, DPTForDepthEstimation

image_processor = AutoImageProcessor.from_pretrained("facebook/dpt-dinov2-small-kitti")
model = DPTForDepthEstimation.from_pretrained("facebook/dpt-dinov2-small-kitti")

In [ ]:
import csv
import cv2
import torch
from transformers import AutoImageProcessor, DPTForDepthEstimation
import numpy as np
import ast
from PIL import Image

In [ ]:
cap = cv2.VideoCapture(<영상위치>)

# DPT 모델 불러오기
image_processor = AutoImageProcessor.from_pretrained("facebook/dpt-dinov2-small-kitti", target_size=(640, 640))
model = DPTForDepthEstimation.from_pretrained("facebook/dpt-dinov2-small-kitti")

depth_maps = []
count = 0
# 프레임 단위로 동영상 읽기
while cap.isOpened():
    count += 1
    print(count)
    # 프레임 단위로 동영상 읽기
    ret, frame = cap.read()
    if not ret:
        break
    height, width, _ = frame.shape

    # 프레임을 모델 입력 형식으로 변환
    inputs = image_processor(frame, return_tensors="pt")

    # DPT 모델에 입력 전달하여 거리 추정
    with torch.no_grad():
        outputs = model(**inputs)  # 변경된 부분
        predicted_depth = outputs.predicted_depth

    # interpolate to original size
    prediction = torch.nn.functional.interpolate(
        predicted_depth.unsqueeze(1),
        size = (height, width),
        mode="bicubic",
        align_corners=False,
    )
    output = prediction.squeeze().cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype("uint8")

    # 추출된 깊이 맵을 리스트에 추가
    depth_maps.append(formatted)

# 종료
cap.release()

# 깊이 맵을 3차원 데이터로 변환하여 저장
depth_maps = np.array(depth_maps)
print("3차원 데이터 형태:", depth_maps.shape)
np.save("dapth_map.npy", depth_maps)


In [ ]:
# npy 파일 로드
loaded_data = np.load("dapth_map.npy")

# 3차원 데이터 확인
print(loaded_data.shape)